# Menu Classifier
## Data from Yelp

Check installed Python versions first
py -0p

Create a virtual environment using Python 3.11 (or 3.12)
py -3.11 -m venv bartinder_env

Activate the virtual environment
.\bartinder_env\Scripts\activate

Now install TensorFlow
pip install --upgrade pip
pip install tensorflow matplotlib kagglehub notebook

 url: https://www.kaggle.com/datasets/joshiatri/yelp-dataset-photos?resource=download


### Step 1: Install required libraries

In [ ]:
#%pip install tensorflow 
%pip install pandas numpy seaborn matplotlib kagglehub

### Step 2: Import Libraries

In [1]:
# Step 2: Import Libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
import kagglehub
import zipfile
import os
import shutil
import json
from PIL import Image

### Step 3: Define paths


In [2]:
# folder_path = r'C:\Users\leomo\Desktop\APP BARTINDER\Menu_Classifier\data'
base_path = r'C:\Users\leomo\Desktop\APP BARTINDER\Menu_Classifier\data\archive'

train_json = os.path.join(base_path, 'train.json')
test_json = os.path.join(base_path, 'test.json')

train_image_path = os.path.join(base_path, 'train', 'train')
test_image_path = os.path.join(base_path, 'test', 'test')

output_data_path = r'C:\Users\leomo\Desktop\APP BARTINDER\Menu_Classifier\data_processed'
os.makedirs(output_data_path, exist_ok=True)

### Step 4: Create folder structure automatically

In [3]:
for dataset in ['train', 'test']:
    for category in ['menu', 'non_menu']:
        os.makedirs(os.path.join(output_data_path, dataset, category), exist_ok=True)


def organize_images(json_file, source_folder, dest_folder):
    with open(json_file, 'r', encoding='utf-8') as file:
        for line in file:
            data = json.loads(line.strip())
            label = data['label']
            photo_id = data['photo_id']
            
            category = 'menu' if label.lower() == 'menu' else 'non_menu'
            
            src = os.path.join(source_folder, f"{photo_id}.jpg")
            dst = os.path.join(dest_folder, category, f"{photo_id}.jpg")
            
            if os.path.exists(src):
                shutil.copy(src, dst)
            else:
                print(f"File {src} not found.")

### Step 5: Function to move images into organized folders

In [4]:

# Organize train images
organize_images(train_json, train_image_path, os.path.join(output_data_path, 'train'))

# Organize test images
organize_images(test_json, test_image_path, os.path.join(output_data_path, 'test'))


# Train your Deep Learning model

In [5]:
train_dir = os.path.join(output_data_path, 'train')
test_dir = os.path.join(output_data_path, 'test')

In [8]:
# Data Generators
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.1)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

Found 57847 images belonging to 2 classes.
Found 6427 images belonging to 2 classes.


In [9]:
# Model setup (MobileNetV2)
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze base layers
base_model.trainable = False

# Add custom classification head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

In [10]:
# Compile
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train
history = model.fit(
    train_generator,
    epochs=5,  # Increase if performance isn't enough
    validation_data=validation_generator
)




Epoch 1/5


 190/1808 [==>...........................] - ETA: 15:26 - loss: 0.0334 - accuracy: 0.9878

UnknownError: Graph execution error:

Detected at node PyFunc defined at (most recent call last):
<stack traces unavailable>
UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x000001DE1A7B23E0>
Traceback (most recent call last):

  File "C:\Users\leomo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tensorflow\python\ops\script_ops.py", line 270, in __call__
    ret = func(*args)
          ^^^^^^^^^^^

  File "C:\Users\leomo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tensorflow\python\autograph\impl\api.py", line 643, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^

  File "C:\Users\leomo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tensorflow\python\data\ops\from_generator_op.py", line 198, in genera

In [ ]:
# Evaluate on Test Set
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

loss, accuracy = model.evaluate(test_generator)
print("Test accuracy:", accuracy)

# Save the model
model.save("../menu_classifier_model.h5")